# 第2章: グラフの制御フロー

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### ライブラリのインストール

このセルは、LangGraphおよび関連するLangChainライブラリをインストールします。実行には数分かかる場合があります。
ご利用になるLLMプロバイダーに応じて、コメントアウトを解除して必要なライブラリをインストールしてください。

In [ ]:
# === ライブラリのインストール ===
# ご利用になるLLMプロバイダーに応じて、以下のコメントを解除して実行してください。
!%pip install -qU langchain langgraph langchain_openai langchain_google_vertexai langchain_google_genai langchain_anthropic langchain_aws boto3 ipython_genutils

### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`

In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**一般的な手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。

In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")

### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。

In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-1.5-flash-001", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0, convert_system_message_to_human=True)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = getattr(llm, 'model', None) or \ 
                 getattr(llm, 'model_name', None) or \ 
                 getattr(llm, 'model_id', None) or \ 
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---

この章では、条件付きエッジやループなど、グラフの流れを制御する方法を学びます。

### ■ 問題001: シンプルなカウンターによるループ

指定された回数だけ特定の処理を繰り返す、基本的なループ構造を構築します。状態にカウンターを持ち、カウンターが上限に達するまでノードの実行を繰り返します。

In [ ]:
# 解答欄001
from typing import TypedDict, Annotated, Union
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class CounterState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int
    max_count: int

# --- ノード定義 (Nodes) ---
def increment_counter(state: CounterState):
    # カウンターを1増やすノード
    current_count = state.get("counter", 0) + 1
    print(f"increment_counter: カウンター = {current_count}")
    return {"messages": [AIMessage(content=f"カウント {current_count}")], "counter": ____}

def process_data(state: CounterState):
    # 何らかの処理を行うノード（ここではメッセージを追加するだけ）
    count = state["counter"]
    print(f"process_data: 現在のカウント {count} で処理を実行中...")
    return {"messages": [AIMessage(content=f"処理 {count} を実行しました。")]}

# --- 条件付きエッジのルーター関数 ---
def should_continue(state: CounterState):
    # カウンターが上限に達したかどうかを判定する
    if state["counter"] < ____:
        print(f"should_continue: ループ継続 (カウンター: {state['counter']}, 上限: {state['max_count']})")
        return "continue_loop" # ループを続ける場合の次のノード
    else:
        print(f"should_continue: ループ終了 (カウンター: {state['counter']}, 上限: {state['max_count']})")
        return "end_loop" # ループを終了する場合

# --- グラフ構築 (Graph) ---
workflow = StateGraph(____)

# ノードの追加
workflow.add_node("entry_node", ____)
workflow.add_node("processing_node", ____)

# エントリポイントの設定
workflow.set_entry_point(____)

# 通常のエッジ
workflow.add_edge("entry_node", ____)

# 条件付きエッジの追加 (ループまたは終了)
workflow.add_conditional_edges(
    "processing_node", # 遷移元のノード
    should_continue,   # ルーター関数
    {
        "continue_loop": ____, # ルーター関数の戻り値とノード名のマッピング
        "end_loop": END
    }
)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- カウンターループのテスト (上限3回) ---")
initial_input = {"messages": [HumanMessage(content="ループ開始")], "max_count": 3, "counter": 0}
for s in app.stream(____):
    print(s)

final_state = app.invoke(____)
print(f"Final State: {final_state}")

print("\n--- カウンターループのテスト (上限1回) ---")
initial_input_short = {"messages": [HumanMessage(content="短いループ開始")], "max_count": 1, "counter": 0}
for s in app.stream(____):
    print(s)

final_state_short = app.invoke(____)
print(f"Final State (Short Loop): {final_state_short}")


<details><summary>解答001</summary>

``````python
# 解答001
from typing import TypedDict, Annotated, Union
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class CounterState(TypedDict):
    messages: Annotated[list, add_messages]
    counter: int
    max_count: int

# --- ノード定義 (Nodes) ---
def increment_counter(state: CounterState):
    # カウンターを1増やすノード
    current_count = state.get("counter", 0) + 1
    print(f"increment_counter: カウンター = {current_count}")
    return {"messages": [AIMessage(content=f"カウント {current_count}")], "counter": current_count}

def process_data(state: CounterState):
    # 何らかの処理を行うノード（ここではメッセージを追加するだけ）
    count = state["counter"]
    print(f"process_data: 現在のカウント {count} で処理を実行中...")
    return {"messages": [AIMessage(content=f"処理 {count} を実行しました。")]}

# --- 条件付きエッジのルーター関数 ---
def should_continue(state: CounterState):
    # カウンターが上限に達したかどうかを判定する
    if state["counter"] < state["max_count"]:
        print(f"should_continue: ループ継続 (カウンター: {state['counter']}, 上限: {state['max_count']})")
        return "continue_loop" # ループを続ける場合の次のノード (entry_nodeへ戻る)
    else:
        print(f"should_continue: ループ終了 (カウンター: {state['counter']}, 上限: {state['max_count']})")
        return "end_loop" # ループを終了する場合 (ENDへ)

# --- グラフ構築 (Graph) ---
workflow = StateGraph(CounterState)

# ノードの追加
workflow.add_node("entry_node", increment_counter)
workflow.add_node("processing_node", process_data)

# エントリポイントの設定
workflow.set_entry_point("entry_node")

# 通常のエッジ
workflow.add_edge("entry_node", "processing_node")

# 条件付きエッジの追加 (ループまたは終了)
workflow.add_conditional_edges(
    "processing_node", # 遷移元のノード
    should_continue,   # ルーター関数
    {
        "continue_loop": "entry_node", # ルーター関数の戻り値とノード名のマッピング
        "end_loop": END
    }
)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- カウンターループのテスト (上限3回) ---")
initial_input = {"messages": [HumanMessage(content="ループ開始")], "max_count": 3, "counter": 0}
for s in app.stream(initial_input):
    print(s)

final_state = app.invoke(initial_input)
print(f"Final State: {final_state}")

print("\n--- カウンターループのテスト (上限1回) ---")
initial_input_short = {"messages": [HumanMessage(content="短いループ開始")], "max_count": 1, "counter": 0}
for s in app.stream(initial_input_short):
    print(s)

final_state_short = app.invoke(initial_input_short)
print(f"Final State (Short Loop): {final_state_short}")
``````
</details>

<details><summary>解説001</summary>

#### この問題のポイント
*   **学習内容:** `add_conditional_edges` を使用して、ノードから自身または他のノードへループバックする基本的なループ構造の作り方を学びます。状態(`State`)にカウンターとループの上限回数を保持し、ルーター関数でループを継続するか終了するかを判断します。
*   **コード解説:**
    *   `CounterState` には、メッセージ履歴(`messages`)に加え、現在のカウント数を保持する `counter` と、ループの最大回数を指定する `max_count` を定義します。
    *   `increment_counter` ノード: グラフのループサイクルの開始点です。現在の `counter` を1増やし、状態を更新します。
    *   `process_data` ノード: ループ内で実行したい主処理を行うノードです。ここでは単純にメッセージを追加しています。
    *   `should_continue` ルーター関数: `processing_node` の後に呼び出されます。`counter` が `max_count` 未満であれば `"continue_loop"` を返し、`entry_node` に戻ってループを継続します。そうでなければ `"end_loop"` を返し、グラフは `END` に遷移して終了します。
    *   `workflow.add_conditional_edges` の設定: `processing_node` から `should_continue` 関数の結果に応じて、`"continue_loop"` なら `entry_node` へ、`"end_loop"` なら `END` へと分岐するように定義しています。これがループ構造の核となります。
    *   グラフ実行時には、`initial_input` で `max_count` と初期 `counter` (通常は0) を設定します。
---
</details>

### ■ 問題002: LLMによるループ継続判断（自己反省ループ）

LLM自身が生成した内容を評価し、改善が必要であればループして再生成を行う「自己反省ループ」を構築します。LLMが特定のキーワード（例：「完璧」）を生成するまで、または指定回数ループするまで処理を繰り返します。

In [ ]:
# 解答欄002
from typing import TypedDict, Annotated, Union, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate

# --- 状態定義 (State) ---
class SelfReflectionState(TypedDict):
    messages: Annotated[list, add_messages]
    current_iteration: int
    max_iterations: int
    target_keyword: str
    current_answer: Optional[str] # LLMの最新の回答を保持

# --- ノード定義 (Nodes) ---
def generate_answer_node(state: SelfReflectionState):
    # LLMに回答を生成させるノード
    iteration = state.get("current_iteration", 0) + 1
    print(f"generate_answer_node: イテレーション {iteration}")
    
    # プロンプトテンプレートの準備
    if iteration == 1:
        prompt_text = "日本の首都はどこですか？簡潔に答えてください。"
    else:
        previous_answer = state.get("current_answer", "")
        prompt_text = f"前回の回答「{previous_answer}」は不十分でした。もっと良い回答を生成してください。特に「完璧」というキーワードを含めるように努力してください。日本の首都はどこですか？"
        
    # LLM呼び出し
    # 実際には、state["messages"]にコンテキストを追加してLLMに渡すが、ここではシンプルにする
    response = llm.invoke([HumanMessage(content=prompt_text)])
    answer_content = response.content
    print(f"generate_answer_node: LLMの回答「{answer_content}」")
    
    return {
        "messages": [AIMessage(content=answer_content)], 
        "current_iteration": ____, 
        "current_answer": ____
    }

# --- 条件付きエッジのルーター関数 ---
def should_reflect_or_finish(state: SelfReflectionState):
    # LLMの回答を評価し、ループを継続するか終了するかを判断する
    current_answer = state.get("current_answer", "")
    current_iteration = state["current_iteration"]
    max_iterations = state["max_iterations"]
    target_keyword = state["target_keyword"]
    
    print(f"should_reflect_or_finish: 現在の回答「{current_answer}」、イテレーション {current_iteration}/{max_iterations}")
    
    if ____ in current_answer:
        print(f"should_reflect_or_finish: ターゲットキーワード「{target_keyword}」を発見。ループ終了。")
        return "finish" # 終了条件
    elif current_iteration >= ____:
        print(f"should_reflect_or_finish: 最大イテレーション {max_iterations} に到達。ループ終了。")
        return "finish" # 終了条件 (最大回数超過)
    else:
        print("should_reflect_or_finish: 改善が必要。ループ継続。")
        return "reflect" # ループ継続条件

# --- グラフ構築 (Graph) ---
workflow = StateGraph(____)

# ノードの追加
workflow.add_node("generator", ____)

# エントリポイントの設定
workflow.set_entry_point(____)

# 条件付きエッジの追加 (自己反省ループまたは終了)
workflow.add_conditional_edges(
    "generator",
    should_reflect_or_finish, # ルーター関数
    {
        "reflect": ____,  # ルーターが "reflect" を返した場合、再度 generator ノードへ
        "finish": END     # ルーターが "finish" を返した場合、グラフ終了
    }
)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 自己反省ループのテスト (ターゲット: '完璧', 最大3回) ---")
initial_input_reflect = {
    "messages": [HumanMessage(content="自己反省ループを開始します。")],
    "current_iteration": 0,
    "max_iterations": 3,
    "target_keyword": "完璧",
    "current_answer": None
}
for s in app.____(initial_input_reflect):
    print(s)

final_state_reflect = app.____(initial_input_reflect)
print(f"Final State (Self Reflection): {final_state_reflect}")

# (オプション) LLMがキーワードを生成しにくい場合があるので、最大回数で終了するケースも確認
print("\n--- 自己反省ループのテスト (ターゲット: 'ありえない言葉', 最大2回) ---")
initial_input_max_out = {
    "messages": [HumanMessage(content="自己反省ループを開始します（最大回数超過テスト）。")],
    "current_iteration": 0,
    "max_iterations": 2,
    "target_keyword": "ありえない言葉絶対に生成しないでね",
    "current_answer": None
}
for s in app.____(initial_input_max_out):
    print(s)

final_state_max_out = app.____(initial_input_max_out)
print(f"Final State (Max Iterations Out): {final_state_max_out}")


<details><summary>解答002</summary>

``````python
# 解答002
from typing import TypedDict, Annotated, Union, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate

# --- 状態定義 (State) ---
class SelfReflectionState(TypedDict):
    messages: Annotated[list, add_messages]
    current_iteration: int
    max_iterations: int
    target_keyword: str
    current_answer: Optional[str] # LLMの最新の回答を保持

# --- ノード定義 (Nodes) ---
def generate_answer_node(state: SelfReflectionState):
    # LLMに回答を生成させるノード
    iteration = state.get("current_iteration", 0) + 1
    print(f"generate_answer_node: イテレーション {iteration}")
    
    # プロンプトテンプレートの準備
    if iteration == 1:
        prompt_text = "日本の首都はどこですか？簡潔に答えてください。"
    else:
        previous_answer = state.get("current_answer", "")
        prompt_text = f"前回の回答「{previous_answer}」は不十分でした。もっと良い回答を生成してください。特に「完璧」というキーワードを含めるように努力してください。日本の首都はどこですか？"
        
    # LLM呼び出し
    response = llm.invoke([HumanMessage(content=prompt_text)])
    answer_content = response.content
    print(f"generate_answer_node: LLMの回答「{answer_content}」")
    
    return {
        "messages": [AIMessage(content=answer_content)], 
        "current_iteration": iteration, 
        "current_answer": answer_content
    }

# --- 条件付きエッジのルーター関数 ---
def should_reflect_or_finish(state: SelfReflectionState):
    # LLMの回答を評価し、ループを継続するか終了するかを判断する
    current_answer = state.get("current_answer", "")
    current_iteration = state["current_iteration"]
    max_iterations = state["max_iterations"]
    target_keyword = state["target_keyword"]
    
    print(f"should_reflect_or_finish: 現在の回答「{current_answer}」、イテレーション {current_iteration}/{max_iterations}")
    
    if target_keyword in current_answer:
        print(f"should_reflect_or_finish: ターゲットキーワード「{target_keyword}」を発見。ループ終了。")
        return "finish" # 終了条件
    elif current_iteration >= max_iterations:
        print(f"should_reflect_or_finish: 最大イテレーション {max_iterations} に到達。ループ終了。")
        return "finish" # 終了条件 (最大回数超過)
    else:
        print("should_reflect_or_finish: 改善が必要。ループ継続。")
        return "reflect" # ループ継続条件

# --- グラフ構築 (Graph) ---
workflow = StateGraph(SelfReflectionState)

# ノードの追加
workflow.add_node("generator", generate_answer_node)

# エントリポイントの設定
workflow.set_entry_point("generator")

# 条件付きエッジの追加 (自己反省ループまたは終了)
workflow.add_conditional_edges(
    "generator",
    should_reflect_or_finish, # ルーター関数
    {
        "reflect": "generator",  # ルーターが "reflect" を返した場合、再度 generator ノードへ
        "finish": END          # ルーターが "finish" を返した場合、グラフ終了
    }
)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 自己反省ループのテスト (ターゲット: '完璧', 最大3回) ---")
initial_input_reflect = {
    "messages": [HumanMessage(content="自己反省ループを開始します。")],
    "current_iteration": 0,
    "max_iterations": 3,
    "target_keyword": "完璧",
    "current_answer": None
}
for s in app.stream(initial_input_reflect):
    print(s)

final_state_reflect = app.invoke(initial_input_reflect)
print(f"Final State (Self Reflection): {final_state_reflect}")

# (オプション) LLMがキーワードを生成しにくい場合があるので、最大回数で終了するケースも確認
print("\n--- 自己反省ループのテスト (ターゲット: 'ありえない言葉', 最大2回) ---")
initial_input_max_out = {
    "messages": [HumanMessage(content="自己反省ループを開始します（最大回数超過テスト）。")],
    "current_iteration": 0,
    "max_iterations": 2,
    "target_keyword": "ありえない言葉絶対に生成しないでね",
    "current_answer": None
}
for s in app.stream(initial_input_max_out):
    print(s)

final_state_max_out = app.invoke(initial_input_max_out)
print(f"Final State (Max Iterations Out): {final_state_max_out}")
``````
</details>

<details><summary>解説002</summary>

#### この問題のポイント
*   **学習内容:** LLMの出力を評価し、その結果に基づいてループを制御する方法を学びます。具体的には、LLMが特定のキーワードを含む回答を生成するまで、または最大試行回数に達するまで、LLMに回答の再生成を指示するループを作成します。
*   **コード解説:**
    *   `SelfReflectionState` には、現在のイテレーション回数 `current_iteration`、最大イテレーション回数 `max_iterations`、目標とするキーワード `target_keyword`、そしてLLMの最新の回答を保存する `current_answer` を追加します。
    *   `generate_answer_node` ノード: LLMを呼び出して回答を生成します。イテレーション回数に応じてプロンプトの内容を変え、2回目以降は前回の回答をフィードバックとして与え、より良い回答（特に `target_keyword` を含む回答）を促します。生成された回答と更新されたイテレーション回数を状態に保存します。
    *   `should_reflect_or_finish` ルーター関数: `generate_answer_node` の後に呼び出されます。以下の条件で次の遷移先を決定します。
        1.  `current_answer` に `target_keyword` が含まれていれば、`"finish"` を返して `END` に遷移します。
        2.  `current_iteration` が `max_iterations` 以上であれば、同様に `"finish"` を返して `END` に遷移します（ループの無限化を防ぐため）。
        3.  上記以外の場合は、`"reflect"` を返し、再度 `generator` ノード（`generate_answer_node`）に戻って処理を繰り返します。
    *   グラフの実行時には、これらの状態を初期設定します。LLMの出力は確率的なため、必ずしも指定回数内にキーワードが生成されるとは限りません。そのため、最大試行回数による終了条件も重要です。
---
</details>

### ■ 問題003: エラーハンドリングとリトライ処理

外部API呼び出しなど、失敗する可能性のある処理をグラフに組み込む場合、エラーハンドリングとリトライ処理は不可欠です。ここでは、模擬的な「不安定なAPI呼び出し」ノードを作成し、それが失敗した場合に指定回数リトライするループを実装します。

In [ ]:
# 解答欄003
import random
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage, BaseMessage

# --- 状態定義 (State) ---
class ErrorHandlingState(TypedDict):
    messages: Annotated[list, add_messages]
    api_call_attempts: int
    max_api_attempts: int
    api_data: Optional[str] # APIから取得したデータ
    error_message: Optional[str] # エラー発生時のメッセージ

# --- ノード定義 (Nodes) ---
def unstable_api_call_node(state: ErrorHandlingState):
    # 模擬的な不安定なAPI呼び出し
    attempts = state.get("api_call_attempts", 0) + 1
    print(f"unstable_api_call_node: API呼び出し試行 {attempts}回目")
    
    # 70%の確率で成功し、30%の確率で失敗すると仮定
    if random.random() < ____: # 成功確率 (例: 0.7)
        print("unstable_api_call_node: API呼び出し成功！")
        api_result = "APIから取得した重要なデータです。"
        return {
            "messages": [AIMessage(content="API呼び出しに成功しました。")],
            "api_call_attempts": ____,
            "api_data": ____,
            "error_message": None # 成功時はエラーメッセージをクリア
        }
    else:
        print("unstable_api_call_node: API呼び出し失敗...")
        error_msg = f"API呼び出し失敗 (試行 {attempts}回目)"
        return {
            "messages": [AIMessage(content=error_msg)],
            "api_call_attempts": ____,
            "api_data": None, # 失敗時はデータをクリア
            "error_message": ____
        }

def error_handler_node(state: ErrorHandlingState):
    # API呼び出しが最終的に失敗した場合の処理
    error_msg = state.get("error_message", "不明なエラーが発生しました。")
    print(f"error_handler_node: {error_msg} 最大リトライ回数に達しました。")
    return {"messages": [AIMessage(content=f"処理失敗: {error_msg}")]}

# --- 条件付きエッジのルーター関数 ---
def should_retry_or_fail(state: ErrorHandlingState):
    # API呼び出しの結果と試行回数に基づいてリトライするか、エラー処理に進むかを判断
    api_data = state.get("api_data")
    attempts = state["api_call_attempts"]
    max_attempts = state["max_api_attempts"]
    
    if state.get(____) is not None: # api_data に何か値があれば成功とみなす
        print("should_retry_or_fail: API成功。処理終了へ。")
        return "succeed" # 成功ルート
    elif attempts < ____: # max_attempts
        print(f"should_retry_or_fail: API失敗。リトライします ({attempts}/{max_attempts})。")
        return "retry" # リトライルート
    else:
        print(f"should_retry_or_fail: API失敗。最大試行回数 ({max_attempts}) に到達。エラー処理へ。")
        return "fail" # 失敗ルート (エラーハンドラへ)

# --- グラフ構築 (Graph) ---
workflow = StateGraph(____)

# ノードの追加
workflow.add_node("api_caller", ____)
workflow.add_node("error_handler", ____)

# エントリポイントの設定
workflow.set_entry_point(____)

# 条件付きエッジの追加 (リトライ、成功、または最終失敗)
workflow.add_conditional_edges(
    "api_caller",
    should_retry_or_fail, # ルーター関数
    {
        "retry": ____,    # "retry" の場合、再度 api_caller ノードへ
        "succeed": END,   # "succeed" の場合、グラフ終了
        "fail": ____      # "fail" の場合、error_handler ノードへ
    }
)

# エラーハンドラノードからの終了
workflow.add_edge(____, END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- エラーハンドリングとリトライ処理のテスト (最大3回試行) ---")
initial_input_retry = {
    "messages": [HumanMessage(content="不安定なAPI呼び出しを開始します。")],
    "api_call_attempts": 0,
    "max_api_attempts": 3,
    "api_data": None,
    "error_message": None
}

# 複数回実行して、成功するケースと失敗するケースを確認
for i in range(5):
    print(f"\n--- 実行試行 {i+1} ---")
    # 状態をリセットするために、毎回新しい入力辞書を作成 (ただし、内容は同じで良い)
    current_input = initial_input_retry.copy() # shallow copyで十分
    # stream実行時の状態変化を追跡するために、api_call_attemptsは都度リセットされる前提
    # invokeの場合は入力が毎回同じなら結果も同じになるが、streamなら途中経過が見れる
    # ここではinvokeを使って最終結果のみ確認するが、streamで途中経過を見るのも有効
    final_state_retry = app.invoke(current_input, {"recursion_limit": 10})
    print(f"Final State (Attempt {i+1}): {final_state_retry}")
    if final_state_retry.get("api_data"):
        print(f"実行試行 {i+1}: API呼び出し成功！ データ: {final_state_retry['api_data']}")
    else:
        print(f"実行試行 {i+1}: API呼び出し最終失敗。エラー: {final_state_retry.get('error_message')}")


<details><summary>解答003</summary>

``````python
# 解答003
import random
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage, BaseMessage

# --- 状態定義 (State) ---
class ErrorHandlingState(TypedDict):
    messages: Annotated[list, add_messages]
    api_call_attempts: int
    max_api_attempts: int
    api_data: Optional[str] # APIから取得したデータ
    error_message: Optional[str] # エラー発生時のメッセージ

# --- ノード定義 (Nodes) ---
def unstable_api_call_node(state: ErrorHandlingState):
    # 模擬的な不安定なAPI呼び出し
    attempts = state.get("api_call_attempts", 0) + 1
    print(f"unstable_api_call_node: API呼び出し試行 {attempts}回目")
    
    # 70%の確率で成功し、30%の確率で失敗すると仮定
    if random.random() < 0.7:
        print("unstable_api_call_node: API呼び出し成功！")
        api_result = "APIから取得した重要なデータです。"
        return {
            "messages": [AIMessage(content="API呼び出しに成功しました。")],
            "api_call_attempts": attempts,
            "api_data": api_result,
            "error_message": None # 成功時はエラーメッセージをクリア
        }
    else:
        print("unstable_api_call_node: API呼び出し失敗...")
        error_msg = f"API呼び出し失敗 (試行 {attempts}回目)"
        return {
            "messages": [AIMessage(content=error_msg)],
            "api_call_attempts": attempts,
            "api_data": None, # 失敗時はデータをクリア
            "error_message": error_msg
        }

def error_handler_node(state: ErrorHandlingState):
    # API呼び出しが最終的に失敗した場合の処理
    error_msg = state.get("error_message", "不明なエラーが発生しました。")
    print(f"error_handler_node: {error_msg} 最大リトライ回数に達しました。")
    return {"messages": [AIMessage(content=f"処理失敗: {error_msg}")]}

# --- 条件付きエッジのルーター関数 ---
def should_retry_or_fail(state: ErrorHandlingState):
    # API呼び出しの結果と試行回数に基づいてリトライするか、エラー処理に進むかを判断
    api_data = state.get("api_data")
    attempts = state["api_call_attempts"]
    max_attempts = state["max_api_attempts"]
    
    if api_data is not None: # api_data に何か値があれば成功とみなす
        print("should_retry_or_fail: API成功。処理終了へ。")
        return "succeed" # 成功ルート
    elif attempts < max_attempts:
        print(f"should_retry_or_fail: API失敗。リトライします ({attempts}/{max_attempts})。")
        return "retry" # リトライルート
    else:
        print(f"should_retry_or_fail: API失敗。最大試行回数 ({max_attempts}) に到達。エラー処理へ。")
        return "fail" # 失敗ルート (エラーハンドラへ)

# --- グラフ構築 (Graph) ---
workflow = StateGraph(ErrorHandlingState)

# ノードの追加
workflow.add_node("api_caller", unstable_api_call_node)
workflow.add_node("error_handler", error_handler_node)

# エントリポイントの設定
workflow.set_entry_point("api_caller")

# 条件付きエッジの追加 (リトライ、成功、または最終失敗)
workflow.add_conditional_edges(
    "api_caller",
    should_retry_or_fail, # ルーター関数
    {
        "retry": "api_caller",    # "retry" の場合、再度 api_caller ノードへ
        "succeed": END,          # "succeed" の場合、グラフ終了
        "fail": "error_handler" # "fail" の場合、error_handler ノードへ
    }
)

# エラーハンドラノードからの終了
workflow.add_edge("error_handler", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- エラーハンドリングとリトライ処理のテスト (最大3回試行) ---")
initial_input_retry = {
    "messages": [HumanMessage(content="不安定なAPI呼び出しを開始します。")],
    "api_call_attempts": 0,
    "max_api_attempts": 3,
    "api_data": None,
    "error_message": None
}

# 複数回実行して、成功するケースと失敗するケースを確認
for i in range(5):
    print(f"\n--- 実行試行 {i+1} ---")
    # 状態をリセットするために、毎回新しい入力辞書を作成 (ただし、内容は同じで良い)
    # LangGraphのinvoke/streamは入力状態を変更しないため、ループ内で同じinputを使い回しても問題ない。
    # ただし、api_call_attemptsはグラフ内で更新されるため、グラフの実行ごとに初期化された状態で開始される。
    final_state_retry = app.invoke(initial_input_retry.copy(), {"recursion_limit": 10})
    print(f"Final State (Attempt {i+1}): {final_state_retry}")
    if final_state_retry.get("api_data"):
        print(f"実行試行 {i+1}: API呼び出し成功！ データ: {final_state_retry['api_data']}")
    else:
        print(f"実行試行 {i+1}: API呼び出し最終失敗。エラー: {final_state_retry.get('error_message')}")
``````
</details>

<details><summary>解説003</summary>

#### この問題のポイント
*   **学習内容:** 失敗する可能性のある処理（ここでは模擬API呼び出し）に対して、リトライ処理を組み込む方法を学びます。状態に試行回数と最大試行回数を保持し、ルーター関数でリトライ、成功、最終失敗の3つのパスに分岐させます。
*   **コード解説:**
    *   `ErrorHandlingState` には、API呼び出しの試行回数 `api_call_attempts`、最大試行回数 `max_api_attempts`、APIから取得したデータ `api_data`、エラーメッセージ `error_message` を定義します。
    *   `unstable_api_call_node` ノード: `random.random()` を使って、確率的に成功または失敗するAPI呼び出しをシミュレートします。成功時は `api_data` に結果を格納し、失敗時は `error_message` を設定します。どちらの場合も `api_call_attempts` を更新します。
    *   `error_handler_node` ノード: リトライ上限に達してもAPI呼び出しが成功しなかった場合に呼び出され、最終的なエラー処理（ここではエラーメッセージの表示）を行います。
    *   `should_retry_or_fail` ルーター関数: `unstable_api_call_node` の後に呼び出されます。
        1.  `api_data` が存在すれば（API呼び出し成功）、`"succeed"` を返して `END` に遷移します。
        2.  `api_data` が存在せず、かつ `api_call_attempts` が `max_api_attempts` 未満であれば、`"retry"` を返し、再度 `api_caller` ノード（`unstable_api_call_node`）に戻ってリトライします。
        3.  上記以外（`api_data` が存在せず、試行回数が上限に達した）の場合は、`"fail"` を返し、`error_handler` ノードに遷移します。
    *   グラフの実行: `random` を使用しているため、実行ごとに結果が変わる可能性があります。複数回実行することで、成功するケース、数回のリトライ後に成功するケース、最終的に失敗するケースを確認できます。`recursion_limit` は、グラフの最大遷移回数を設定するもので、意図しない無限ループを防ぐために重要です。
---
</details>

### ■ 問題004: Human-in-the-Loop (人間による介在)

LangGraphでは、`Interrupt` を使用してグラフの実行を一時停止し、人間の確認や入力を待つことができます。この問題では、LLMがメールの下書きを作成した後、ユーザーが内容を確認・承認するまで処理を中断し、承認後に次のステップ（ここでは「送信」のシミュレーション）に進むグラフを構築します。

In [ ]:
# 解答欄004
from typing import TypedDict, Annotated, Optional, List
from langgraph.graph import StateGraph, END, Interrupt
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from IPython.display import Image, display
import uuid

# --- 状態定義 (State) ---
class HumanApprovalState(TypedDict):
    messages: Annotated[list, add_messages]
    draft_email: Optional[str] # LLMが生成したメール下書き
    user_feedback: Optional[str] # ユーザーからのフィードバックや承認 ("approve" または修正指示)
    is_approved: Optional[bool] # 承認されたかどうか

# --- ノード定義 (Nodes) ---
def draft_email_node(state: HumanApprovalState):
    # LLMにメールの下書きを生成させるノード
    print("draft_email_node: メール下書きを生成中...")
    # ユーザーの最初の要望、または修正指示に基づいて下書きを作成
    if state.get("user_feedback") and not state.get("is_approved"):
        # 修正指示がある場合
        instruction = f"前回のメール下書き「{state['draft_email']}」に対して、以下の修正指示があります。\n\n修正指示: {state['user_feedback']}\n\nこの指示に基づいてメール下書きを修正してください。"
    else:
        # 初回の下書き作成
        instruction = state["messages"][-1].content # 最初のユーザーメッセージ
    
    prompt = f"以下の指示に基づいて、丁寧なビジネスメールの下書きを作成してください。\n\n指示: {instruction}\n\n下書き:"
    response = llm.invoke([HumanMessage(content=prompt)])
    email_draft = response.content
    print(f"draft_email_node: 生成された下書き「{email_draft}」")
    
    # ユーザーフィードバックと承認状態をリセット
    return {
        "messages": [AIMessage(content=f"下書きを作成しました。ご確認ください。\n---\n{email_draft}\n---")], 
        "draft_email": ____, 
        "user_feedback": None, 
        "is_approved": False # 確認待ちなのでFalseに
    }

def send_email_node(state: HumanApprovalState):
    # メール送信をシミュレートするノード
    final_email = state.get("draft_email", "")
    print(f"send_email_node: メールを送信します...\n内容:\n{final_email}")
    return {"messages": [AIMessage(content=f"メールを送信しました。\n内容：{final_email}")]}

def request_human_approval_node(state: HumanApprovalState):
    # ユーザーに承認を求めるノード（Interruptの前に配置）
    print("request_human_approval_node: ユーザーの承認待ちです。Interruptが発生します。承認する場合は 'approve' と入力してください。修正する場合は修正内容を記述してください。")
    # このノードは状態を直接変更せず、Interruptが続くことを示す
    return {}

# --- 条件付きエッジのルーター関数 ---
def route_after_approval_request(state: HumanApprovalState):
    # ユーザーのフィードバックに基づいて次のアクションを決定
    # このルーターはInterruptの後に呼び出される想定
    user_input = state.get("user_feedback", "").strip().lower()
    print(f"route_after_approval_request: ユーザー入力「{user_input}」")
    
    if user_input == "approve" and state.get("is_approved"):
        print("route_after_approval_request: 承認されました。メール送信へ。")
        return "send" # 承認ルート
    elif user_input and user_input != "approve": # 何か入力があり、それが 'approve' でなければ修正とみなす
        print("route_after_approval_request: 修正指示がありました。下書き修正へ。")
        return "revise" # 修正ルート (再度draft_email_nodeへ)
    else: # 入力が空など、予期せぬ場合は再度確認を求める
        print("route_after_approval_request: 不明な入力または未承認。再度確認を求めます。")
        return "re_request" # 再度確認を促すルート

# --- グラフ構築 (Graph) ---
workflow = StateGraph(____)

# ノードの追加
workflow.add_node("draft_email", ____)
workflow.add_node("request_approval", ____)
workflow.add_node("send_email", ____)

# エントリポイントの設定
workflow.set_entry_point(____)

# 通常のエッジ
workflow.add_edge(____, "request_approval") # 下書き作成後、承認依頼へ

# 承認依頼ノードの後にInterruptを挟む
# `request_approval` ノードが完了すると、Interruptが発生し、人間からの入力を待つ
# 人間が `app.update_state(config, {"user_feedback": "approve"})` のように状態を更新して再開すると、
# `route_after_approval_request` が呼び出される
workflow.add_conditional_edges(
    "request_approval",
    route_after_approval_request,
    {
        "send": ____,
        "revise": ____,
        "re_request": ____ # 不明な入力の場合、再度承認依頼へ
    }
)

workflow.add_edge("send_email", END) # 送信後終了

# グラフのコンパイル (Interruptを指定)
# Interruptは、指定されたノードの実行 *後* に発生します。
# ここでは 'request_approval' ノードの後に中断を挿入します。
app = workflow.compile(interrupt_after=[____])

# --- グラフの可視化 (オプション) ---
try:
    img_bytes = app.get_graph().draw_png()
    display(Image(img_bytes))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- グラフの実行と人間による介在のシミュレーション ---
print("\n--- Human-in-the-Loop テスト開始 ---")
initial_email_request = "来週の月曜日に予定しているプロジェクトAに関する進捗会議のリマインダーメールを作成してください。参加者はBさんとCさんです。"
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

# 1. 最初の実行 (下書き作成 -> 承認依頼 -> Interrupt)
print("\n--- ステップ1: 初期リクエストと下書き生成 ---")
events = []
for event in app.stream(
    {"messages": [HumanMessage(content=initial_email_request)]},
    config,
    stream_mode="values"
):
    events.append(event)
    print(f"イベント: {event}")
    # messagesの最後がAIMessageで、かつInterruptの前なら表示
    if isinstance(event.get("messages", [])[-1], AIMessage) and "request_approval" in event:
        print(f"AIからのメッセージ: {event['messages'][-1].content}")

print("\n--- ステップ1完了: グラフは中断状態のはずです。 ---")
assert events[-1]["request_approval"] is not None # request_approvalノードで止まっていることを確認
current_draft = events[-1]["request_approval"]["draft_email"]
print(f"現在のメール下書き:\n{current_draft}")

# 2. 人間が内容を確認し、修正を指示して再開
print("\n--- ステップ2: ユーザーが修正を指示して再開 ---")
user_correction = "会議の場所として第3会議室を追記してください。"
events_after_correction = []
for event in app.stream(
    None, # 入力はNoneで、現在の状態から再開
    # 正しくは app.update_state で状態を更新してから stream(None, config) を呼び出す
    # config={"configurable": {"thread_id": config["configurable"]["thread_id"], "user_feedback": user_correction, "is_approved": False}},
    stream_mode="values"
):
    events_after_correction.append(event)
    print(f"イベント: {event}")
    if isinstance(event.get("messages", [])[-1], AIMessage) and "request_approval" in event:
        print(f"AIからのメッセージ (修正後): {event['messages'][-1].content}")

print("\n--- ステップ2完了: グラフは再度中断状態のはずです。 ---")
assert events_after_correction[-1]["request_approval"] is not None
corrected_draft = events_after_correction[-1]["request_approval"]["draft_email"]
print(f"修正後のメール下書き:\n{corrected_draft}")

# 3. 人間が内容を承認して再開
print("\n--- ステップ3: ユーザーが承認して再開 ---")
events_after_approval = []
for event in app.stream(
    None, # 入力はNoneで、現在の状態から再開
    # 正しくは app.update_state で状態を更新してから stream(None, config) を呼び出す
    # config={"configurable": {"thread_id": config["configurable"]["thread_id"], "user_feedback": "approve", "is_approved": True}},
    stream_mode="values"
):
    events_after_approval.append(event)
    print(f"イベント: {event}")
    if "send_email" in event:
        print(f"AIからのメッセージ (送信後): {event['messages'][-1].content}")

print("\n--- ステップ3完了: グラフは終了しているはずです。 ---")
assert "__end__" in events_after_approval[-1] # 最終的に終了したことを確認
final_sent_email_details = events_after_approval[-1]["__end__"]
print(f"最終送信メール情報: {final_sent_email_details}")


<details><summary>解答004</summary>

``````python
# 解答004
from typing import TypedDict, Annotated, Optional, List
from langgraph.graph import StateGraph, END, Interrupt
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from IPython.display import Image, display
import uuid

# --- 状態定義 (State) ---
class HumanApprovalState(TypedDict):
    messages: Annotated[list, add_messages]
    draft_email: Optional[str] # LLMが生成したメール下書き
    user_feedback: Optional[str] # ユーザーからのフィードバックや承認 ("approve" または修正指示)
    is_approved: Optional[bool] # 承認されたかどうか

# --- ノード定義 (Nodes) ---
def draft_email_node(state: HumanApprovalState):
    # LLMにメールの下書きを生成させるノード
    print("draft_email_node: メール下書きを生成中...")
    # ユーザーの最初の要望、または修正指示に基づいて下書きを作成
    if state.get("user_feedback") and not state.get("is_approved"):
        # 修正指示がある場合 (user_feedbackに修正内容が入っている想定)
        instruction = f"前回のメール下書き「{state['draft_email']}」に対して、以下の修正指示があります。\n\n修正指示: {state['user_feedback']}\n\nこの指示に基づいてメール下書きを修正してください。"
    else:
        # 初回の下書き作成 (messagesの最後のHumanMessageから取得)
        user_request_message = next((msg for msg in reversed(state["messages"]) if isinstance(msg, HumanMessage)), None)
        instruction = user_request_message.content if user_request_message else "一般的なビジネスメールを作成してください。"
    
    prompt = f"以下の指示に基づいて、丁寧なビジネスメールの下書きを作成してください。\n\n指示: {instruction}\n\n下書き:"
    response = llm.invoke([HumanMessage(content=prompt)])
    email_draft = response.content
    print(f"draft_email_node: 生成された下書き「{email_draft}」")
    
    # ユーザーフィードバックと承認状態をリセット（あるいは明確に設定）
    return {
        "messages": [AIMessage(content=f"下書きを作成しました。ご確認ください。\n---\n{email_draft}\n---")], 
        "draft_email": email_draft, 
        "user_feedback": None, # フィードバックは消費されたのでクリア
        "is_approved": False # 新しい下書きなので未承認状態に
    }

def send_email_node(state: HumanApprovalState):
    # メール送信をシミュレートするノード
    final_email = state.get("draft_email", "")
    print(f"send_email_node: メールを送信します...\n内容:\n{final_email}")
    return {"messages": [AIMessage(content=f"メールを送信しました。\n内容：{final_email}")]}

def request_human_approval_node(state: HumanApprovalState):
    # ユーザーに承認を求めるノード（Interruptの前に配置）
    print("request_human_approval_node: ユーザーの承認待ちです。Interruptが発生します。承認する場合は 'approve' と入力してください。修正する場合は修正内容を記述してください。")
    # このノードは状態を直接変更せず、Interruptが続くことを示す
    return {}

# --- 条件付きエッジのルーター関数 ---
def route_after_approval_request(state: HumanApprovalState):
    # ユーザーのフィードバックに基づいて次のアクションを決定
    # このルーターはInterruptの後に、状態が更新された後に呼び出される
    user_input = state.get("user_feedback", "").strip().lower()
    is_approved_flag = state.get("is_approved", False)
    print(f"route_after_approval_request: ユーザー入力「{user_input}」, 承認状態: {is_approved_flag}")
    
    if is_approved_flag and user_input == "approve":
        print("route_after_approval_request: 承認されました。メール送信へ。")
        return "send" # 承認ルート
    elif user_input and user_input != "approve": # 何か入力があり、それが 'approve' でなければ修正とみなす
        print("route_after_approval_request: 修正指示がありました。下書き修正へ。")
        return "revise" # 修正ルート (再度draft_email_nodeへ)
    else: # 入力が空、または'approve'だがis_approvedフラグがFalseなど、予期せぬ場合は再度確認を求める
        print("route_after_approval_request: 不明な入力または状態の不整合。再度確認を求めます。")
        # 状態をリセットして再要求するのが安全かもしれない
        # state["user_feedback"] = None 
        # state["is_approved"] = False
        return "re_request" # 再度確認を促すルート

# --- グラフ構築 (Graph) ---
workflow = StateGraph(HumanApprovalState)

# ノードの追加
workflow.add_node("draft_email", draft_email_node)
workflow.add_node("request_approval", request_human_approval_node) 
workflow.add_node("send_email", send_email_node)

# エントリポイントの設定
workflow.set_entry_point("draft_email")

# 通常のエッジ
workflow.add_edge("draft_email", "request_approval") # 下書き作成後、承認依頼へ

# 承認依頼ノードの後にInterruptを挟む
# `request_approval` ノードが完了すると、Interruptが発生し、人間からの入力を待つ
# 人間が `app.update_state(config, {"user_feedback": "approve", "is_approved": True})` のように状態を更新して再開すると、
# `route_after_approval_request` が呼び出される
workflow.add_conditional_edges(
    "request_approval", # このノードの後にInterruptが入り、再開時にこのルーターが呼ばれる
    route_after_approval_request,
    {
        "send": "send_email",
        "revise": "draft_email",
        "re_request": "request_approval" # 不明な入力の場合、再度承認依頼へ
    }
)

workflow.add_edge("send_email", END) # 送信後終了

# グラフのコンパイル (Interruptを指定)
# Interruptは、指定されたノードの実行 *後* に発生します。
# ここでは 'request_approval' ノードの後に中断を挿入します。
app = workflow.compile(interrupt_after=["request_approval"])

# --- グラフの可視化 (オプション) ---
try:
    img_bytes = app.get_graph().draw_png()
    display(Image(img_bytes))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- グラフの実行と人間による介在のシミュレーション ---
print("\n--- Human-in-the-Loop テスト開始 ---")
initial_email_request = "来週の月曜日に予定しているプロジェクトAに関する進捗会議のリマインダーメールを作成してください。参加者はBさんとCさんです。"
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

# 1. 最初の実行 (下書き作成 -> 承認依頼 -> Interrupt)
print("\n--- ステップ1: 初期リクエストと下書き生成 ---")
events = []
for event in app.stream(
    {"messages": [HumanMessage(content=initial_email_request)]},
    config,
    stream_mode="values" # valuesモードで各ノードの出力状態を取得
):
    events.append(event)
    # print(f"イベント: {list(event.keys())}") # デバッグ用にキーを表示
    # messagesの最後がAIMessageで、かつInterruptの前なら表示
    last_message = event.get("messages", [])[-1] if event.get("messages") else None
    if isinstance(last_message, AIMessage) and "request_approval" in event:
        print(f"AIからのメッセージ: {last_message.content}")

print("\n--- ステップ1完了: グラフは中断状態のはずです。 ---")
current_state_after_step1 = app.get_state(config) # 中断時の状態を取得
assert "request_approval" in current_state_after_step1.next # 次に実行されるべきノードが request_approval の後の中断点であることを確認
current_draft = current_state_after_step1.values["draft_email"]
print(f"現在のメール下書き:\n{current_draft}")

# 2. 人間が内容を確認し、修正を指示して再開
print("\n--- ステップ2: ユーザーが修正を指示して再開 ---")
user_correction = "会議の場所として第3会議室を追記してください。"
events_after_correction = []
# update_stateではなく、stream/invokeの入力として渡すことで状態を更新
for event in app.stream(
    None, # 入力はNoneで、現在の状態から再開
    # configに直接更新したい状態を含めるのではなく、
    # app.update_state を使うか、あるいは次のノードの入力として渡すのが一般的。
    # ここではデモのため、streamの入力として直接更新値を渡すのではなく、
    # HumanApprovalStateのuser_feedbackとis_approvedを更新した入力で再開するイメージで記述します。
    # LangGraphの stream/invoke は通常、最初の入力のみを受け付け、
    # 2回目以降の呼び出しで `None` 以外の入力を渡すと新しい実行として扱われることがあるため注意。
    # 正しくは、app.update_state を使用するか、グラフが次の入力を期待する設計にする。
    # ここでは、Interrupt後の再開は app.update_state で状態を更新してから app.stream(None, config) が推奨される
    input_after_correction={"user_feedback": user_correction, "is_approved": False}, # この形式は正しくない。update_stateを使うべき
    config=config, # thread_idは同じものを使用
    stream_mode="values"
):
    # 正しい再開方法:
    # app.update_state(config, {"user_feedback": user_correction, "is_approved": False})
    # for event in app.stream(None, config, stream_mode="values"):
    events_after_correction.append(event)
    # print(f"イベント: {list(event.keys())}")
    last_message = event.get("messages", [])[-1] if event.get("messages") else None
    if isinstance(last_message, AIMessage) and "request_approval" in event:
        print(f"AIからのメッセージ (修正後): {last_message.content}")

print("\n--- ステップ2完了: グラフは再度中断状態のはずです。 (正しい実装なら) ---")
# 上記のstream呼び出しは正しくないため、アサーションはコメントアウト
# current_state_after_step2 = app.get_state(config)
# assert "request_approval" in current_state_after_step2.next
# corrected_draft = current_state_after_step2.values["draft_email"]
# print(f"修正後のメール下書き:\n{corrected_draft}")

# 正しい方法でステップ2を再実行
print("\n--- ステップ2 (正しい方法): ユーザーが修正を指示して再開 ---")
app.update_state(config, {"user_feedback": user_correction, "is_approved": False})
events_after_correction_correct = []
for event in app.stream(None, config, stream_mode="values"):
    events_after_correction_correct.append(event)
    # print(f"イベント: {list(event.keys())}")
    last_message = event.get("messages", [])[-1] if event.get("messages") else None
    if isinstance(last_message, AIMessage) and "request_approval" in event:
         print(f"AIからのメッセージ (修正後): {last_message.content}")

current_state_after_step2_correct = app.get_state(config)
assert "request_approval" in current_state_after_step2_correct.next
corrected_draft_correct = current_state_after_step2_correct.values["draft_email"]
print(f"修正後のメール下書き:\n{corrected_draft_correct}")

# 3. 人間が内容を承認して再開
print("\n--- ステップ3: ユーザーが承認して再開 ---")
app.update_state(config, {"user_feedback": "approve", "is_approved": True})
events_after_approval = []
for event in app.stream(None, config, stream_mode="values"):
    events_after_approval.append(event)
    # print(f"イベント: {list(event.keys())}")
    if "send_email" in event and event.get("messages"):
        print(f"AIからのメッセージ (送信後): {event['messages'][-1].content}")

print("\n--- ステップ3完了: グラフは終了しているはずです。 ---")
final_state_after_step3 = app.get_state(config)
assert final_state_after_step3.next == () # nextが空タプルなら終了
final_sent_email_details = final_state_after_step3.values
print(f"最終送信メール情報: {final_sent_email_details}")
``````
</details>

### ■ 問題005: 複数の制御フローの組み合わせ (反復的な改善と最終承認)

これまでに学んだ複数の制御フロー技術（LLMによる判断ループ、人間による介在）を組み合わせて、より実践的なシナリオを構築します。具体的には、LLMが提案を生成し、ユーザーがその提案を評価します。ユーザーが「完璧」と評価するまで、または最大試行回数に達するまでLLMによる改善ループが実行されます。ループが終了した後、最終的な提案について再度ユーザーに承認を求め、承認されれば処理を完了します。

In [ ]:
# 解答欄005
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END, Interrupt
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
import uuid

# --- 状態定義 (State) ---
class IterativeApprovalState(TypedDict):
    messages: Annotated[list, add_messages]
    original_request: str # ユーザーの最初の要求
    current_proposal: Optional[str] # LLMによる現在の提案
    user_critique: Optional[str] # ユーザーからの批判や改善点
    iteration_count: int # 改善ループの現在のイテレーション回数
    max_iterations: int # 改善ループの最大イテレーション回数
    is_proposal_accepted_by_llm: bool # LLMが提案は十分と判断したか (改善ループ終了条件)
    is_final_proposal_approved_by_human: Optional[bool] # 人間による最終承認

# --- ノード定義 (Nodes) ---
def generate_proposal_node(state: IterativeApprovalState):
    # LLMが提案を生成または改善するノード
    iteration = state.get("iteration_count", 0) + 1
    print(f"generate_proposal_node: イテレーション {iteration}")

    if iteration == 1:
        prompt_text = f"以下の要望に基づいて、詳細な提案書を作成してください。\n\n要望: {state['original_request']}"
    else:
        prompt_text = f"前回の提案「{state['current_proposal']}」に対して、ユーザーから以下の改善点が指摘されました。「{state['user_critique']}」。これらの点を踏まえて提案を大幅に改善してください。もし改善点が特にない、または提案が完璧だと思ったら、提案の最後に「提案は完璧です。」と記述してください。"
    
    response = llm.invoke([HumanMessage(content=prompt_text)])
    proposal = response.content
    print(f"generate_proposal_node: 生成された提案「{proposal}」")
    
    accepted_by_llm = "提案は完璧です。" in proposal
    
    return {
        "messages": [AIMessage(content=proposal)],
        "current_proposal": ____,
        "iteration_count": ____,
        "is_proposal_accepted_by_llm": ____,
        "user_critique": None # 消費したのでクリア
    }

def request_human_critique_node(state: IterativeApprovalState):
    # ユーザーに提案の評価（批判や改善点）を求めるノード (改善ループ用Interrupt)
    print("request_human_critique_node: 生成された提案について評価を入力してください。改善ループを終了し最終承認に進む場合は '完璧' と入力してください。")
    # Interruptはこのノードの後に発生
    return {}

def request_final_approval_node(state: IterativeApprovalState):
    # 最終提案についてユーザーに承認を求めるノード (最終承認用Interrupt)
    proposal = state.get("current_proposal", "")
    print(f"request_final_approval_node: 最終提案が完成しました。ご確認ください。承認する場合は 'approve' と入力してください。\n---\n{proposal}\n---")
    # Interruptはこのノードの後に発生
    return {}

def process_complete_node(state: IterativeApprovalState):
    # 全てのプロセスが完了したことを示すノード
    print("process_complete_node: 提案は最終承認され、プロセスは完了しました。")
    return {"messages": [AIMessage(content="提案は最終承認され、プロセスは完了しました。")]}

# --- 条件付きエッジのルーター関数 ---
def route_after_critique(state: IterativeApprovalState):
    # ユーザーの評価とLLMの判断に基づいて改善ループを継続するか、最終承認に進むかを判断
    user_critique = state.get("user_critique", "").strip().lower()
    iteration = state["iteration_count"]
    max_iter = state["max_iterations"]
    llm_thinks_perfect = state.get("is_proposal_accepted_by_llm", False)

    print(f"route_after_critique: ユーザー評価「{user_critique}」, LLM判断「{llm_thinks_perfect}」, イテレーション {iteration}/{max_iter}")

    if user_critique == "完璧" or ____ or ____ >= ____:
        print("route_after_critique: 改善ループ終了。最終承認へ。")
        return "proceed_to_final_approval" # 最終承認へ
    else:
        print("route_after_critique: 改善が必要。改善ループ継続。")
        return "continue_improvement" # 改善ループ継続 (generate_proposal_nodeへ)

def route_after_final_approval(state: IterativeApprovalState):
    # ユーザーの最終承認に基づいてプロセスを完了するか、再度検討を促す（ここでは簡略化）
    # このルーターはfinal_approval_request_nodeの後のInterruptの後に呼び出される
    final_approval_feedback = state.get("user_critique", "").strip().lower() # user_critiqueを最終承認のフィードバックにも使う
    print(f"route_after_final_approval: 最終承認フィードバック「{final_approval_feedback}」")

    if final_approval_feedback == "approve" and state.get("is_final_proposal_approved_by_human"):
        print("route_after_final_approval: 最終承認。プロセス完了へ。")
        return "complete_process" # プロセス完了へ
    else:
        # ここでは簡略化のため、非承認の場合も終了するが、実際は最初のプロセスに戻るなどの処理も考えられる
        print("route_after_final_approval: 最終的に非承認。プロセス終了（要改善）。")
        return "end_process_unapproved" # プロセス終了 (ENDへ)

# --- グラフ構築 (Graph) ---
workflow = StateGraph(____)

# ノードの追加
workflow.add_node("generate_proposal", ____)
workflow.add_node("request_critique", ____)
workflow.add_node("request_final_approval", ____)
workflow.add_node("process_complete", ____)

# エントリポイント
workflow.set_entry_point(____)

# 改善ループのフロー
workflow.add_edge(____, "request_critique") # 提案生成後、人間による評価依頼へ

workflow.add_conditional_edges(
    "request_critique", # このノードの後にInterruptが入り、再開時にこのルーターが呼ばれる
    ____, # route_after_critique
    {
        "continue_improvement": ____, # "generate_proposal"
        "proceed_to_final_approval": ____ # "request_final_approval"
    }
)

# 最終承認フロー
workflow.add_conditional_edges(
    "request_final_approval", # このノードの後にInterruptが入り、再開時にこのルーターが呼ばれる
    ____, # route_after_final_approval
    {
        "complete_process": ____, # "process_complete"
        "end_process_unapproved": ____ # END
    }
)

workflow.add_edge(____, END) # "process_complete"

# グラフのコンパイル (複数のInterruptポイントを指定)
app = workflow.compile(interrupt_after=[____, ____]) # "request_critique", "request_final_approval"

# --- グラフの可視化 (オプション) ---
try:
    img_bytes = app.get_graph().draw_png()
    display(Image(img_bytes))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}")

# --- グラフ実行シミュレーション ---
print("\n--- 複合制御フローテスト開始 ---")
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

initial_user_request = "新しいエコフレンドリーなコーヒーカップのデザイン案を作成してください。ターゲットは若年層で、持ち運びやすさが重要です。"

# ステップ1: 最初の提案生成 -> 人間による評価依頼 (Interrupt)
print("\n--- 改善ループ ステップ1: 最初の提案と評価依頼 ---")
current_state = app.invoke(
    {
        "messages": [HumanMessage(content=initial_user_request)],
        "original_request": initial_user_request,
        "iteration_count": 0,
        "max_iterations": 3,
        "is_proposal_accepted_by_llm": False,
    },
    config
)
print(f"AIからの提案(1回目):\n{current_state['current_proposal']}")
assert current_state["messages"][-1].type == "ai" # AIからのメッセージがあるはず
assert app.get_state(config).next == ("request_critique",) # request_critiqueの後で止まっているはず

# ステップ2: 人間が改善点を入力 -> LLMが改善案を生成 -> 再度評価依頼 (Interrupt)
print("\n--- 改善ループ ステップ2: ユーザーが改善点を入力し、LLMが改善 ---")
app.update_state(config, {"user_critique": "もっと具体的な素材の提案と、ユニークな形状のアイデアを加えてください。"})
current_state = app.invoke(None, config) # configを渡して再開
print(f"AIからの提案(2回目):\n{current_state['current_proposal']}")
assert app.get_state(config).next == ("request_critique",)

# ステップ3: 人間が「完璧」と評価 -> 改善ループ終了 -> 最終承認依頼 (Interrupt)
print("\n--- 改善ループ ステップ3: ユーザーが「完璧」と評価し、最終承認へ ---")
app.update_state(config, {"user_critique": "完璧"})
current_state = app.invoke(None, config)
print(f"最終承認前の提案:\n{current_state['current_proposal']}")
assert app.get_state(config).next == ("request_final_approval",) # request_final_approvalの後で止まっているはず

# ステップ4: 人間が最終承認 -> プロセス完了 (END)
print("\n--- 最終承認ステップ: ユーザーが最終承認 ---")
app.update_state(config, {"user_critique": "approve", "is_final_proposal_approved_by_human": True})
final_state = app.invoke(None, config)
print(f"最終状態のメッセージ: {final_state['messages'][-1].content}")
assert app.get_state(config).next == () # グラフが終了している
print("\n--- 複合制御フローテスト完了 ---")

# (オプション) 最大イテレーションで改善ループが終了するケース
print("\n--- 複合制御フローテスト (最大イテレーションケース) ---")
thread_id_max_iter = str(uuid.uuid4())
config_max_iter = {"configurable": {"thread_id": thread_id_max_iter}}
current_state_max_iter = app.invoke(
    {
        "messages": [HumanMessage(content=initial_user_request)],
        "original_request": initial_user_request,
        "iteration_count": 0,
        "max_iterations": 1, # 最大1イテレーションに設定
        "is_proposal_accepted_by_llm": False,
    },
    config_max_iter
)
print(f"AIからの提案(1回目):\n{current_state_max_iter['current_proposal']}")
app.update_state(config_max_iter, {"user_critique": "もっとカラフルにできないか？"}) # 完璧とは言わない
current_state_max_iter = app.invoke(None, config_max_iter)
print(f"最大イテレーション後の提案:\n{current_state_max_iter['current_proposal']}") 
# この時点で iteration_count が max_iterations に達しているので、次は final_approval に進むはず
assert app.get_state(config_max_iter).next == ("request_final_approval",)
print("最大イテレーションで改善ループが終了し、最終承認に進むことを確認。")
app.update_state(config_max_iter, {"user_critique": "approve", "is_final_proposal_approved_by_human": True})
final_state_max_iter = app.invoke(None, config_max_iter)
assert app.get_state(config_max_iter).next == () 
print("最大イテレーションケースも正常に完了。")


<details><summary>解答005</summary>

``````python
# 解答005
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END, Interrupt
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from IPython.display import Image, display
import uuid

# --- 状態定義 (State) ---
class IterativeApprovalState(TypedDict):
    messages: Annotated[list, add_messages]
    original_request: str # ユーザーの最初の要求
    current_proposal: Optional[str] # LLMによる現在の提案
    user_critique: Optional[str] # ユーザーからの批判や改善点 (改善ループと最終承認の両方で利用)
    iteration_count: int # 改善ループの現在のイテレーション回数
    max_iterations: int # 改善ループの最大イテレーション回数
    is_proposal_accepted_by_llm: bool # LLMが提案は十分と判断したか (改善ループ終了条件)
    is_final_proposal_approved_by_human: Optional[bool] # 人間による最終承認

# --- ノード定義 (Nodes) ---
def generate_proposal_node(state: IterativeApprovalState):
    # LLMが提案を生成または改善するノード
    iteration = state.get("iteration_count", 0) + 1
    print(f"generate_proposal_node: イテレーション {iteration}")

    if iteration == 1:
        prompt_text = f"以下の要望に基づいて、詳細な提案書を作成してください。\n\n要望: {state['original_request']}"
    else:
        prompt_text = f"前回の提案「{state['current_proposal']}」に対して、ユーザーから以下の改善点が指摘されました。「{state['user_critique']}」。これらの点を踏まえて提案を大幅に改善してください。もし改善点が特にない、または提案が完璧だと思ったら、提案の最後に「提案は完璧です。」と記述してください。"
    
    response = llm.invoke([HumanMessage(content=prompt_text)])
    proposal = response.content
    print(f"generate_proposal_node: 生成された提案「{proposal}」")
    
    accepted_by_llm = "提案は完璧です。" in proposal
    
    return {
        "messages": [AIMessage(content=proposal)],
        "current_proposal": proposal,
        "iteration_count": iteration,
        "is_proposal_accepted_by_llm": accepted_by_llm,
        "user_critique": None # 消費したのでクリア
    }

def request_human_critique_node(state: IterativeApprovalState):
    # ユーザーに提案の評価（批判や改善点）を求めるノード (改善ループ用Interrupt)
    print("request_human_critique_node: 生成された提案について評価を入力してください。改善ループを終了し最終承認に進む場合は '完璧' と入力してください。")
    # Interruptはこのノードの後に発生
    return {}

def request_final_approval_node(state: IterativeApprovalState):
    # 最終提案についてユーザーに承認を求めるノード (最終承認用Interrupt)
    proposal = state.get("current_proposal", "")
    print(f"request_final_approval_node: 最終提案が完成しました。ご確認ください。承認する場合は 'approve' と入力してください。\n---\n{proposal}\n---")
    # Interruptはこのノードの後に発生
    return {}

def process_complete_node(state: IterativeApprovalState):
    # 全てのプロセスが完了したことを示すノード
    print("process_complete_node: 提案は最終承認され、プロセスは完了しました。")
    return {"messages": [AIMessage(content="提案は最終承認され、プロセスは完了しました。")]}

# --- 条件付きエッジのルーター関数 ---
def route_after_critique(state: IterativeApprovalState):
    # ユーザーの評価とLLMの判断に基づいて改善ループを継続するか、最終承認に進むかを判断
    user_critique = state.get("user_critique", "").strip().lower()
    iteration = state["iteration_count"]
    max_iter = state["max_iterations"]
    llm_thinks_perfect = state.get("is_proposal_accepted_by_llm", False)

    print(f"route_after_critique: ユーザー評価「{user_critique}」, LLM判断「{llm_thinks_perfect}」, イテレーション {iteration}/{max_iter}")

    if user_critique == "完璧" or llm_thinks_perfect or iteration >= max_iter:
        print("route_after_critique: 改善ループ終了。最終承認へ。")
        return "proceed_to_final_approval" # 最終承認へ
    else:
        print("route_after_critique: 改善が必要。改善ループ継続。")
        return "continue_improvement" # 改善ループ継続 (generate_proposal_nodeへ)

def route_after_final_approval(state: IterativeApprovalState):
    # ユーザーの最終承認に基づいてプロセスを完了するか、再度検討を促す（ここでは簡略化）
    # このルーターはfinal_approval_request_nodeの後のInterruptの後に呼び出される
    final_approval_feedback = state.get("user_critique", "").strip().lower() # user_critiqueを最終承認のフィードバックにも使う
    is_human_approved = state.get("is_final_proposal_approved_by_human", False)
    print(f"route_after_final_approval: 最終承認フィードバック「{final_approval_feedback}」, 人間承認フラグ: {is_human_approved}")

    if final_approval_feedback == "approve" and is_human_approved:
        print("route_after_final_approval: 最終承認。プロセス完了へ。")
        return "complete_process" # プロセス完了へ
    else:
        # ここでは簡略化のため、非承認の場合も終了するが、実際は最初のプロセスに戻るなどの処理も考えられる
        print("route_after_final_approval: 最終的に非承認または不正な状態。プロセス終了（要改善）。")
        return "end_process_unapproved" # プロセス終了 (ENDへ)

# --- グラフ構築 (Graph) ---
workflow = StateGraph(IterativeApprovalState)

# ノードの追加
workflow.add_node("generate_proposal", generate_proposal_node)
workflow.add_node("request_critique", request_human_critique_node)
workflow.add_node("request_final_approval", request_final_approval_node)
workflow.add_node("process_complete", process_complete_node)

# エントリポイント
workflow.set_entry_point("generate_proposal")

# 改善ループのフロー
workflow.add_edge("generate_proposal", "request_critique") # 提案生成後、人間による評価依頼へ

workflow.add_conditional_edges(
    "request_critique", # このノードの後にInterruptが入り、再開時にこのルーターが呼ばれる
    route_after_critique,
    {
        "continue_improvement": "generate_proposal", # 改善を続ける場合
        "proceed_to_final_approval": "request_final_approval" # 最終承認に進む場合
    }
)

# 最終承認フロー
workflow.add_conditional_edges(
    "request_final_approval", # このノードの後にInterruptが入り、再開時にこのルーターが呼ばれる
    route_after_final_approval,
    {
        "complete_process": "process_complete",
        "end_process_unapproved": END 
    }
)

workflow.add_edge("process_complete", END)

# グラフのコンパイル (複数のInterruptポイントを指定)
app = workflow.compile(interrupt_after=["request_critique", "request_final_approval"])

# --- グラフの可視化 (オプション) ---
try:
    img_bytes = app.get_graph().draw_png()
    display(Image(img_bytes))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}")

# --- グラフ実行シミュレーション ---
print("\n--- 複合制御フローテスト開始 ---")
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

initial_user_request = "新しいエコフレンドリーなコーヒーカップのデザイン案を作成してください。ターゲットは若年層で、持ち運びやすさが重要です。"

# ステップ1: 最初の提案生成 -> 人間による評価依頼 (Interrupt)
print("\n--- 改善ループ ステップ1: 最初の提案と評価依頼 ---")
current_state = app.invoke(
    {
        "messages": [HumanMessage(content=initial_user_request)],
        "original_request": initial_user_request,
        "iteration_count": 0,
        "max_iterations": 3,
        "is_proposal_accepted_by_llm": False,
        "is_final_proposal_approved_by_human": None # 初期化
    },
    config
)
print(f"AIからの提案(1回目):\n{current_state['current_proposal']}")
assert current_state["messages"][-1].type == "ai" # AIからのメッセージがあるはず
assert app.get_state(config).next == ("request_critique",) # request_critiqueの後で止まっているはず

# ステップ2: 人間が改善点を入力 -> LLMが改善案を生成 -> 再度評価依頼 (Interrupt)
print("\n--- 改善ループ ステップ2: ユーザーが改善点を入力し、LLMが改善 ---")
app.update_state(config, {"user_critique": "もっと具体的な素材の提案と、ユニークな形状のアイデアを加えてください。"})
current_state = app.invoke(None, config) # configを渡して再開
print(f"AIからの提案(2回目):\n{current_state['current_proposal']}")
assert app.get_state(config).next == ("request_critique",)

# ステップ3: 人間が「完璧」と評価 -> 改善ループ終了 -> 最終承認依頼 (Interrupt)
print("\n--- 改善ループ ステップ3: ユーザーが「完璧」と評価し、最終承認へ ---")
app.update_state(config, {"user_critique": "完璧"})
current_state = app.invoke(None, config)
print(f"最終承認前の提案:\n{current_state['current_proposal']}")
assert app.get_state(config).next == ("request_final_approval",) # request_final_approvalの後で止まっているはず

# ステップ4: 人間が最終承認 -> プロセス完了 (END)
print("\n--- 最終承認ステップ: ユーザーが最終承認 ---")
app.update_state(config, {"user_critique": "approve", "is_final_proposal_approved_by_human": True})
final_state = app.invoke(None, config)
print(f"最終状態のメッセージ: {final_state['messages'][-1].content}")
assert app.get_state(config).next == () # グラフが終了している
print("\n--- 複合制御フローテスト完了 ---")

# (オプション) 最大イテレーションで改善ループが終了するケース
print("\n--- 複合制御フローテスト (最大イテレーションケース) ---")
thread_id_max_iter = str(uuid.uuid4())
config_max_iter = {"configurable": {"thread_id": thread_id_max_iter}}
current_state_max_iter = app.invoke(
    {
        "messages": [HumanMessage(content=initial_user_request)],
        "original_request": initial_user_request,
        "iteration_count": 0,
        "max_iterations": 1, # 最大1イテレーションに設定
        "is_proposal_accepted_by_llm": False,
        "is_final_proposal_approved_by_human": None
    },
    config_max_iter
)
print(f"AIからの提案(1回目):\n{current_state_max_iter['current_proposal']}")
app.update_state(config_max_iter, {"user_critique": "もっとカラフルにできないか？"}) # 完璧とは言わない
current_state_max_iter = app.invoke(None, config_max_iter)
print(f"最大イテレーション後の提案:\n{current_state_max_iter['current_proposal']}") 
# この時点で iteration_count が max_iterations に達しているので、次は final_approval に進むはず
assert app.get_state(config_max_iter).next == ("request_final_approval",)
print("最大イテレーションで改善ループが終了し、最終承認に進むことを確認。")
app.update_state(config_max_iter, {"user_critique": "approve", "is_final_proposal_approved_by_human": True})
final_state_max_iter = app.invoke(None, config_max_iter)
assert app.get_state(config_max_iter).next == () 
print("最大イテレーションケースも正常に完了。")
``````
</details>